In [ ]:
import pandas as pd
import os

# Пути к файлам и соответствующие названия колонок
file_paths = {
    'bert': '/bert/sim-words5-bert.csv',
    'glove': '/glove_python/sim-words10-glove.csv',
    'pmi': '/pmi/similar_words_pmi.csv',
    'word2vec': '/word2vec/similar_words10.csv'
}

# Создаем пустой DataFrame с колонкой 'word'
data = pd.DataFrame(columns=['word'])

# Обрабатываем каждый файл
for model_name, path in file_paths.items():
    # Читаем CSV-файл
    full_path = ".." + path
    df = pd.read_csv(full_path)
    
    # Переименовываем колонку Most_Similar_Word в название модели
    df = df.rename(columns={'Most_Similar_Word': model_name})
    
    # Если это первый файл, используем его слова как основу
    if data.empty:
        data['word'] = df['Word']
    
    # Добавляем данные в основной DataFrame
    data[model_name] = df[model_name]

In [ ]:
import ast
# Функция для проверки, является ли значение пустым списком или NaN
def is_valid_similar_words(value):
    if pd.isna(value):  # проверяем NaN
        return False
    if isinstance(value, str):  # если данные хранятся как строки (например, "[('слово', 0.5), ...]")
        try:
            lst = ast.literal_eval(value)  # преобразуем строку в список
            return len(lst) > 0  # True, если список не пустой
        except (ValueError, SyntaxError):
            return False
    elif isinstance(value, list):  # если данные уже в формате списка
        return len(value) > 0
    else:
        return False  # на случай других форматов

# Применяем фильтрацию: оставляем строки, где ВСЕ столбцы с похожими словами не пусты
filtered_data = data[
    data['bert'].apply(is_valid_similar_words) &
    data['glove'].apply(is_valid_similar_words) &
    data['pmi'].apply(is_valid_similar_words) &
    data['word2vec'].apply(is_valid_similar_words)
]

Исходное количество строк: 71796
Отфильтрованное количество строк: 17058
           word                                               bert  \
0           маи  [('раи', 0.84766555), ('рами', 0.8392539), ('л...   
1    состояться  [('завершиться', 0.86279786), ('начаться', 0.8...   
2  общенародныи  [('всенародныи', 0.9310285), ('чужеродныи', 0....   
4   бессмертныи  [('ратныи', 0.88773453), ('смертныи', 0.881054...   
5          полк                            [('полка', 0.82291174)]   

                                               glove  \
0  [('февраль', 0.9127440452575684), ('ноябрь', 0...   
1  [('пройти', 0.9210653901100159), ('октябрь', 0...   
2  [('организатор', 0.6860509514808655), ('органи...   
4  [('бессмертный', 0.9052135944366455), ('шестви...   
5  [('участие', 0.9297742247581482), ('принимать'...   

                                                 pmi  \
0                     [('полк', 0.8976994641215542)]   
1              [('бессмертный', 0.8976994641215542)]   
2

In [9]:
n = 100  # кол-во лучайных строк

random_sample = filtered_data.sample(n=n, random_state=42)  # random_state для воспроизводимости

# Сохраняем в новый CSV-файл
output_file = 'random_sample_similar_words.csv'
random_sample.to_csv(output_file, index=False, encoding='utf-8')